In [197]:
import json

In [575]:
class Board:

    # Class Variable
    type = '9x9'

    box_map = {(0, 0) : 0,
    (0, 1) : 1,
    (0, 2) : 2,
    (1, 0) : 3,
    (1, 1) : 4,
    (1, 2) : 5,
    (2, 0) : 6,
    (2, 1) : 7,
    (2, 2) : 8 }

    # The init method or constructor
    def __init__(self, chunk):
        lines = chunk.split('\n')
        # Instance Variables
        self.orig_board = chunk
        self.name = lines[0]
        self.board_map = {}
        rows = list(map(list, lines[1:10]))      
        # self._init_board_map(rows)
        self.empty_squares = {}
        self._calc_empties(rows)
        self.solve_log = []
        
        
        self.update_candidate_notes()

    def __str__(self):
        st = f"{self.name}\n{self.get_rows()}"
        return st.replace('],', '],\n')

    # def _init_board_map(self, rows):
        
    def _calc_empties(self, rows):
        for r in range(0,9):
            row = rows[r] 
            for c in range(0,9):
                self.board_map[(r,c, self.get_box_for_square( (r,c) ))] = row[c]
                if row[c] == '0':
                    self.empty_squares[(r,c)] = None

    def calc_candidates(self, square):
        (r,c) = square
        cands = set(list('123456789'))
        b = self.get_box_for_square(square)
        cands = cands.difference(self.get_row(r))
        cands = cands.difference(self.get_col(c))
        cands = cands.difference(self.get_box(self.get_box_for_square(square)))
        self.empty_squares[square] = cands
        # if len(cands) > 0:
        #     self.empty_squares[square] = cands
        # else:
        #     del self.empty_squares[square]
            
    def set_square(self, square, val):
        (r, c) = square
        # self.get_row(r)[c] = val
        # self.rows[r][c] = val
        self.board_map[(r, c, self.get_box_for_square((r,c)) )] = val
        if (r,c) in self.empty_squares:
            # self.empty_squares.pop( (r,c))
            del self.empty_squares[ (r,c)]

    def get_row(self, n):
        # return self.rows[n]
        return [val for key, val in self.board_map.items() if key[0]==n]

    def get_col(self, n):
        return [val for key, val in self.board_map.items() if key[1]==n]

    def get_box(self, n, flat_list=True):
        return [val for key, val in self.board_map.items() if key[2]==n]
            
    def get_rows(self):
        # return self.rows
        return [b.get_row(r) for r in range(0,9)]

    def get_cols(self):
        return list(map(lambda x: self.get_col(x), range(0,9)))
        
    def get_boxes(self):
        return list(map(lambda x: self.get_box(x), range(0,9)))
        
    def get_box_row(self, n):
        if n==0:
            return self.get_rows()[0:3]
        if n==1:
            return self.get_rows()[3:6]
        if n==2:
            return self.get_rows()[6:9]
            
        return 'No way Jose'
        
    def get_box_col(self, n):
        return [self.get_col(0), self.get_col(1), self.get_col(2)]

    def get_box_for_square(self, square):
        # return self.get_box(0)
        (r, c) = square
        b_row = r//3
        b_col = c//3
        # print(f'box: {(b_row, b_col)}')
        return Board.box_map[(b_row, b_col)]

    def get_rows_containing_digit(self, digit):
        return [i for i, x in enumerate(map(lambda x: digit in x, self.get_rows())) if x]

    def get_cols_containing_digit(self, digit):
        return [i for i, x in enumerate(map(lambda x: digit in x, self.get_cols())) if x]

    def get_boxes_containing_digit(self, digit):
        return [i for i, x in enumerate(map(lambda x: digit in x, self.get_boxes())) if x]

    def update_candidate_notes(self):
        for sq in self.empty_squares:
            self.calc_candidates(sq)
        self.solve_log.append(f'{len(self.empty_squares)} squares open. Candidates per square summary: {self.summarize_board()}')

    def get_solved_squares(self):
        return {key : val for key, val in self.empty_squares.items() if len(val)==1}

    def get_two_candidate_squares(self):
        return {key : val for key, val in self.empty_squares.items() if len(val)==2}
        
    def fill_in_solved_squares(self):
        single_candidates = self.get_solved_squares()
        for sq in single_candidates:
            self.set_square(sq, single_candidates[sq].pop())
            # self.empty_squares.pop(sq)

    def summarize_board(self):
        return hash_count(list(map(len, self.empty_squares.values())), sort=True)

    def forward_solve(self):
        while len(self.empty_squares) > 0:
            self.fill_in_solved_squares()
            self.update_candidate_notes()
            # for i in range(0,9):
            #     self.check_row(i)
            #     self.check_col(i)
            if len(self.get_solved_squares()) == 0:
                break 

    def apply_solves(self, rounds=5):
        self.forward_solve()
        if len(self.empty_squares) == 0:
            return True
        
        last_empty_count = len(self.empty_squares)

        for idx in range(0,rounds):
            # print(idx, len(self.empty_squares))
            for i in range(0,9):
                self.check_row(i)
                self.check_col(i)
            self.forward_solve()
            # if len(self.empty_squares) == last_empty_count:
            #     return False
            # last_empty_count = self.empty_squares
            
    def check_row(self, r):
        # solutions of type: last remaining spot for digit
        j = filter_for_row(self.empty_squares, r)
        digit_spots={}
        for k, v in j.items():
            for m in v:
                if m in digit_spots:
                    digit_spots[m].append(k[1])
                else:
                    digit_spots[m] = [k[1]]
                    
        solutions = [(v[0],k) for k,v in digit_spots.items() if len(v)==1]
        if solutions:
            for k,v in solutions:
                sq = (r,k)
                # print(f'digit: {v} can only go in {sq}')
                self.set_square(sq, v)

    def check_col(self, c):
        # solutions of type: last remaining spot for digit
        j = filter_for_col(self.empty_squares, c)
        digit_spots={}
        for k, v in j.items():
            for m in v:
                if m in digit_spots:
                    digit_spots[m].append(k[0])
                else:
                    digit_spots[m] = [k[0]]
                    
        solutions = [(v[0],k) for k,v in digit_spots.items() if len(v)==1]
    
        if solutions:
            for k,v in solutions:
                sq = (k,c)
                # print(f'digit: {v} can only go in {sq}')
                self.set_square(sq, v)
    
    def check_col0(self):
        # get everything from boxCol0, ie. boxes 0,3,6, left side of board
        bmap = {key : val for key, val in self.board_map.items() if key[2] in [0, 3, 6]}
        # check digits which appear twice in the group of 3 boxes
        # i.e. just need one more, in 3rd box
        return {key : val for key, val in invert_map( bmap ).items() if len(val)==2}

    def export_boardstring(self):
        # same format as original input file
        return f'{self.name}\n' + '\n'.join(list(map(lambda r: ''.join(r), self.get_rows())))
        
    def export_svg_board(self, filename=None, show_notes=False):
        if filename:
            outname = filename
        else:
            outname = f'{self.name}.svg'
            
        with open(outname, 'w') as outfile:
            with open('board_template_basefile.svg', 'r') as infile:
                outfile.write(infile.read())
                for sq in self.board_map:
                    (row,col,box) = sq
                    if self.board_map[sq] != '0':
                        svg_sq = f'<text x="{100*(col+1)}" y="{100*(row+1)}">{b.board_map[sq]}</text>'
                        outfile.write(svg_sq)
                    elif show_notes:
                        if self.empty_squares[row,col]:
                            d={}
                            for e in self.empty_squares[(row,col)]:
                                d[e]=e
                            svg_sq = f'''
    <text x="{100*(col+1)}" y="{100*(row+1)-40}" class="note">{d.get('1', '&#160;')} {d.get('2', '&#160;')} {d.get('3', '&#160;')}</text>
    <text x="{100*(col+1)}" y="{100*(row+1)-10}" class="note">{d.get('4', '&#160;')} {d.get('5', '&#160;')} {d.get('6', '&#160;')}</text>
    <text x="{100*(col+1)}" y="{100*(row+1)+20}" class="note">{d.get('7', '&#160;')} {d.get('8', '&#160;')} {d.get('9', '&#160;')}</text>
    '''
                            outfile.write(svg_sq)
                        
                outfile.write('</g></svg>')

def flatten(matrix):
    return [item for row in matrix for item in row]

def filter_for_row(dct, row):
    return dict([(k,v) for k, v in dct.items() if k[0]==row])

def filter_for_col(dct, col):
    return dict([(k,v) for k, v in dct.items() if k[1]==col])

def hash_count(lst, sort=False):
    summary = {}
    for val in lst:
        summary[val] = 1 + summary.get(val, 0)
    if sort:
        return dict(sorted(summary.items()))
    else:
        return summary

def hash_count_bis(lst, sort=False):
    summary = {}
    for val in lst:
        if val[1] in summary:
            summary[val[1]].append(val)
        else:
            summary[val[1]] = [val]
    if sort:
        return dict(sorted(summary.items()))
    else:
        return summary

def invert_map(map):
    inverted_map = {}
    for key, val in map.items():
        if val in inverted_map:
            inverted_map[val].append(key)
        else:
            inverted_map[val] = [key]

    return inverted_map

In [576]:
list('123456789')

['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [577]:
def buildboard(chunk):
    b = {}
    lines = chunk.split('\n')
    b[lines[0]] = list(map(list, lines[1:10]))
    return b

In [578]:
#infile = 'sudoku.txt'
infile = 'p096_sudoku.txt'
boardchunks = []
boardlines = ''
with open(infile) as f:
    for line in f:        
        if line.startswith('Grid') and boardlines:
            boardchunks.append(boardlines)
            boardlines = line
        else:
            boardlines += line
boardchunks

['Grid 01\n003020600\n900305001\n001806400\n008102900\n700000008\n006708200\n002609500\n800203009\n005010300\n',
 'Grid 02\n200080300\n060070084\n030500209\n000105408\n000000000\n402706000\n301007040\n720040060\n004010003\n',
 'Grid 03\n000000907\n000420180\n000705026\n100904000\n050000040\n000507009\n920108000\n034059000\n507000000\n',
 'Grid 04\n030050040\n008010500\n460000012\n070502080\n000603000\n040109030\n250000098\n001020600\n080060020\n',
 'Grid 05\n020810740\n700003100\n090002805\n009040087\n400208003\n160030200\n302700060\n005600008\n076051090\n',
 'Grid 06\n100920000\n524010000\n000000070\n050008102\n000000000\n402700090\n060000000\n000030945\n000071006\n',
 'Grid 07\n043080250\n600000000\n000001094\n900004070\n000608000\n010200003\n820500000\n000000005\n034090710\n',
 'Grid 08\n480006902\n002008001\n900370060\n840010200\n003704100\n001060049\n020085007\n700900600\n609200018\n',
 'Grid 09\n000900002\n050123400\n030000160\n908000000\n070000090\n000000205\n091000050\n00743902

In [587]:
boards={}
for bc in boardchunks:
    b = Board(bc)
    b.forward_solve()
    # for i in range(0,9):
    #     b.check_row(i)
    #     b.check_col(i)
    # # b.forward_solve()
    # # for i in range(0,9):
        
    # # b.forward_solve()
    # b.apply_solves(7)
    boards[b.name] = b
    print(f'{b.name}: {len(b.empty_squares)}')

Grid 01: 0
Grid 02: 50
Grid 03: 50
Grid 04: 47
Grid 05: 0
Grid 06: 57
Grid 07: 53
Grid 08: 0
Grid 09: 55
Grid 10: 53
Grid 11: 52
Grid 12: 0
Grid 13: 37
Grid 14: 54
Grid 15: 30
Grid 16: 0
Grid 17: 0
Grid 18: 47
Grid 19: 0
Grid 20: 0
Grid 21: 49
Grid 22: 44
Grid 23: 39
Grid 24: 41
Grid 25: 52
Grid 26: 55
Grid 27: 50
Grid 28: 42
Grid 29: 54
Grid 30: 54
Grid 31: 54
Grid 32: 49
Grid 33: 49
Grid 34: 0
Grid 35: 46
Grid 36: 0
Grid 37: 49
Grid 38: 0
Grid 39: 45
Grid 40: 0
Grid 41: 55
Grid 42: 55
Grid 43: 53
Grid 44: 57
Grid 45: 53
Grid 46: 57
Grid 47: 58
Grid 48: 56
Grid 49: 59
Grid 50: 54


In [581]:
print(boards['Grid 15'].empty_squares)

{(1, 3): set(), (4, 3): {'7', '3'}, (4, 5): {'7', '3'}, (7, 3): {'7', '3'}, (7, 5): {'7', '3'}}


In [567]:
print(boards['Grid 15'])

Grid 15
[['3', '4', '1', '2', '8', '6', '7', '9', '7'],
 ['8', '9', '5', '1', '0', '7', '4', '6', '3'],
 ['7', '2', '6', '9', '3', '4', '5', '1', '8'],
 ['5', '7', '3', '4', '6', '9', '1', '8', '2'],
 ['9', '6', '8', '5', '2', '1', '7', '3', '4'],
 ['4', '1', '2', '8', '7', '3', '6', '5', '9'],
 ['2', '8', '9', '6', '4', '5', '3', '7', '1'],
 ['6', '3', '1', '7', '9', '2', '8', '4', '5'],
 ['4', '5', '7', '3', '1', '8', '9', '2', '6']]


In [582]:
b = Board(boardchunks[2])
print(b.board_map)

{(0, 0, 0): '0', (0, 1, 0): '0', (0, 2, 0): '0', (0, 3, 1): '0', (0, 4, 1): '0', (0, 5, 1): '0', (0, 6, 2): '9', (0, 7, 2): '0', (0, 8, 2): '7', (1, 0, 0): '0', (1, 1, 0): '0', (1, 2, 0): '0', (1, 3, 1): '4', (1, 4, 1): '2', (1, 5, 1): '0', (1, 6, 2): '1', (1, 7, 2): '8', (1, 8, 2): '0', (2, 0, 0): '0', (2, 1, 0): '0', (2, 2, 0): '0', (2, 3, 1): '7', (2, 4, 1): '0', (2, 5, 1): '5', (2, 6, 2): '0', (2, 7, 2): '2', (2, 8, 2): '6', (3, 0, 3): '1', (3, 1, 3): '0', (3, 2, 3): '0', (3, 3, 4): '9', (3, 4, 4): '0', (3, 5, 4): '4', (3, 6, 5): '0', (3, 7, 5): '0', (3, 8, 5): '0', (4, 0, 3): '0', (4, 1, 3): '5', (4, 2, 3): '0', (4, 3, 4): '0', (4, 4, 4): '0', (4, 5, 4): '0', (4, 6, 5): '0', (4, 7, 5): '4', (4, 8, 5): '0', (5, 0, 3): '0', (5, 1, 3): '0', (5, 2, 3): '0', (5, 3, 4): '5', (5, 4, 4): '0', (5, 5, 4): '7', (5, 6, 5): '0', (5, 7, 5): '0', (5, 8, 5): '9', (6, 0, 6): '9', (6, 1, 6): '2', (6, 2, 6): '0', (6, 3, 7): '1', (6, 4, 7): '0', (6, 5, 7): '8', (6, 6, 8): '0', (6, 7, 8): '0', (6, 8, 

In [583]:
b.name

'Grid 03'

In [584]:
b.apply_solves()

In [585]:
print(b)

Grid 03
[['2', '6', '1', '8', '8', '3', '9', '5', '7'],
 ['7', '9', '5', '4', '2', '6', '1', '8', '3'],
 ['3', '8', '1', '7', '9', '5', '4', '2', '6'],
 ['1', '7', '2', '9', '3', '4', '8', '6', '5'],
 ['6', '5', '9', '2', '6', '1', '7', '4', '2'],
 ['4', '4', '8', '5', '6', '7', '3', '1', '9'],
 ['9', '2', '6', '1', '7', '8', '5', '3', '4'],
 ['8', '3', '4', '6', '5', '9', '7', '7', '1'],
 ['5', '1', '7', '3', '4', '2', '6', '9', '8']]


In [479]:
b.export_svg_board(show_notes=True)

In [478]:
b.forward_solve()

In [463]:
b.get_row(2)

['0', '3', '0', '5', '6', '0', '2', '0', '9']

In [400]:
[(k,v) for k, v in b.empty_squares.items() if k[0]==2]

[((2, 0), {'1', '8'}),
 ((2, 2), {'7', '8'}),
 ((2, 5), {'1', '4'}),
 ((2, 7), {'1', '7'})]

In [476]:
for r in range(0,9):
    b.check_row(r)

digit: 6 can only go in (0, 8)
digit: 4 can only go in (2, 5)
digit: 8 can only go in (5, 1)
digit: 6 can only go in (6, 3)


In [474]:
for c in range(0,9):
    b.check_col(c)

digit: 4 can only go in (0, 1)
digit: 5 can only go in (6, 4)
digit: 6 can only go in (4, 6)


In [401]:
j = filter_for_row(b.empty_squares, 2)
j

{(2, 0): {'1', '8'},
 (2, 2): {'7', '8'},
 (2, 5): {'1', '4'},
 (2, 7): {'1', '7'}}

In [402]:
digit_spots={}
for k, v in j.items():
    for m in v:
        if m in digit_spots:
            digit_spots[m].append(k[1])
        else:
            digit_spots[m] = [k[1]]
digit_spots

{'8': [0, 2], '1': [0, 5, 7], '7': [2, 7], '4': [5]}

In [405]:
[(v[0],k) for k,v in digit_spots.items() if len(v)==1]

[(5, '4')]

In [366]:
print(b)

Grid 01
[['4', '8', '3', '9', '2', '1', '6', '5', '7'],
 ['9', '6', '7', '3', '4', '5', '8', '2', '1'],
 ['2', '5', '1', '8', '7', '6', '4', '9', '3'],
 ['5', '4', '8', '1', '3', '2', '9', '7', '6'],
 ['7', '2', '9', '5', '6', '4', '1', '3', '8'],
 ['1', '3', '6', '7', '9', '8', '2', '4', '5'],
 ['3', '7', '2', '6', '8', '9', '5', '1', '4'],
 ['8', '1', '4', '2', '5', '3', '7', '6', '9'],
 ['6', '9', '5', '4', '1', '7', '3', '8', '2']]


In [368]:
len(b.empty_squares)

0

In [352]:
j = b.check_col0()
j

{'3': [(2, 1, 0), (6, 0, 6)], '4': [(5, 0, 3), (8, 2, 6)]}

In [363]:
for k,v in j.items():
    print(k, {0,3,6}.difference({x[2] for x in v}))

3 {3}
4 {0}


In [314]:
b.set_square( (0,1), '4')

In [315]:
b.set_square( (0,8), '6')

In [316]:
b.set_square( (6,8), '2')

In [320]:
b.set_square( (8,0), '6')

In [323]:
b.set_square( (3,2), '6')

In [326]:
b.set_square( (4,8), '7')

In [329]:
b.set_square( (8,6), '7')

In [330]:
b.forward_solve()

In [333]:
b.solve_log

['51 squares open. Candidates per square summary: {1: 1, 2: 12, 3: 16, 4: 13, 5: 7, 6: 2}',
 '50 squares open. Candidates per square summary: {2: 13, 3: 16, 4: 12, 5: 7, 6: 2}',
 '47 squares open. Candidates per square summary: {1: 1, 2: 13, 3: 17, 4: 9, 5: 5, 6: 2}',
 '46 squares open. Candidates per square summary: {1: 1, 2: 16, 3: 15, 4: 8, 5: 4, 6: 2}',
 '45 squares open. Candidates per square summary: {1: 1, 2: 17, 3: 13, 4: 8, 5: 4, 6: 2}',
 '44 squares open. Candidates per square summary: {2: 17, 3: 13, 4: 9, 5: 3, 6: 2}',
 '43 squares open. Candidates per square summary: {1: 1, 2: 17, 3: 12, 4: 9, 5: 2, 6: 2}',
 '42 squares open. Candidates per square summary: {1: 1, 2: 18, 3: 14, 4: 6, 5: 2, 6: 1}',
 '41 squares open. Candidates per square summary: {2: 20, 3: 13, 4: 6, 5: 1, 6: 1}',
 '40 squares open. Candidates per square summary: {2: 19, 3: 14, 4: 5, 5: 1, 6: 1}',
 '39 squares open. Candidates per square summary: {2: 19, 3: 13, 4: 6, 5: 1}',
 '38 squares open. Candidates per

In [243]:
# b.fill_in_solved_squares()
print(b.solve_log)

['51 squares open. Candidates per square summary: {1: 1, 2: 12, 3: 16, 4: 13, 5: 7, 6: 2}', '50 squares open. Candidates per square summary: {2: 13, 3: 16, 4: 12, 5: 7, 6: 2}', '50 squares open. Candidates per square summary: {1: 1, 2: 13, 3: 15, 4: 13, 5: 6, 6: 2}', '49 squares open. Candidates per square summary: {1: 1, 2: 15, 3: 16, 4: 10, 5: 5, 6: 2}', '48 squares open. Candidates per square summary: {1: 1, 2: 17, 3: 14, 4: 9, 5: 5, 6: 2}', '47 squares open. Candidates per square summary: {2: 17, 3: 14, 4: 10, 5: 4, 6: 2}']


In [190]:
b.update_candidate_notes()
print(b.solve_log)

['51 squares open. Candidates per square summary: {1: 1, 2: 12, 3: 16, 4: 13, 5: 7, 6: 2}', '50 squares open. Candidates per square summary: {2: 13, 3: 16, 4: 12, 5: 7, 6: 2}']


In [191]:
print(b)

Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [192]:
b.forward_solve??

Signature: b.forward_solve()
Docstring: <no docstring>
Source:   
    def forward_solve(self):
        while len(self.empty_squares) > 0:
            self.fill_in_solved_squares()
            self.update_candidate_notes()
            if len(self.get_solved_squares()) == 0:
                break
File:      c:\users\jake hendrickson\appdata\local\temp\ipykernel_14072\521901088.py
Type:      method

In [193]:
b.get_col(8)

['0', '4', '9', '8', '0', '0', '0', '0', '3']

In [21]:
# get everything from boxCol0, ie. boxes 0,3,6, left side of board
bmap = {key : val for key, val in b.board_map.items() if key[2] in [0, 3, 6]}
bmap

{(0, 0, 0): '2',
 (0, 1, 0): '0',
 (0, 2, 0): '0',
 (1, 0, 0): '0',
 (1, 1, 0): '6',
 (1, 2, 0): '0',
 (2, 0, 0): '0',
 (2, 1, 0): '3',
 (2, 2, 0): '0',
 (3, 0, 3): '0',
 (3, 1, 3): '0',
 (3, 2, 3): '0',
 (4, 0, 3): '0',
 (4, 1, 3): '0',
 (4, 2, 3): '0',
 (5, 0, 3): '4',
 (5, 1, 3): '0',
 (5, 2, 3): '2',
 (6, 0, 6): '3',
 (6, 1, 6): '0',
 (6, 2, 6): '1',
 (7, 0, 6): '7',
 (7, 1, 6): '2',
 (7, 2, 6): '0',
 (8, 0, 6): '0',
 (8, 1, 6): '0',
 (8, 2, 6): '4'}

In [22]:
invert_map( bmap )

{'2': [(0, 0, 0), (5, 2, 3), (7, 1, 6)],
 '0': [(0, 1, 0),
  (0, 2, 0),
  (1, 0, 0),
  (1, 2, 0),
  (2, 0, 0),
  (2, 2, 0),
  (3, 0, 3),
  (3, 1, 3),
  (3, 2, 3),
  (4, 0, 3),
  (4, 1, 3),
  (4, 2, 3),
  (5, 1, 3),
  (6, 1, 6),
  (7, 2, 6),
  (8, 0, 6),
  (8, 1, 6)],
 '6': [(1, 1, 0)],
 '3': [(2, 1, 0), (6, 0, 6)],
 '4': [(5, 0, 3), (8, 2, 6)],
 '1': [(6, 2, 6)],
 '7': [(7, 0, 6)]}

In [23]:
# check digits which appear twice in the group of 3 boxes
# i.e. just need one more, in 3rd box
{key : val for key, val in invert_map( bmap ).items() if len(val)==2}

{'3': [(2, 1, 0), (6, 0, 6)], '4': [(5, 0, 3), (8, 2, 6)]}

In [24]:
# For the digit 4, zero in on the one box which needs it, and the one column allowed to hold it
trio_needs_4 = {key : val for key, val in b.board_map.items() if key[2] in [0] and key[1]==1}  # box==0 and column==1
# checking middle column of box0 to see if there's a spot for '4'
trio_needs_4

{(0, 1, 0): '0', (1, 1, 0): '6', (2, 1, 0): '3'}

In [25]:
check_trio_values_4 = invert_map(trio_needs_4)
check_trio_values_4

{'0': [(0, 1, 0)], '6': [(1, 1, 0)], '3': [(2, 1, 0)]}

In [26]:
## 4 needs to go someplace in this set of 3 cells (i.e. trio)
## Check if there is only a single blank remaining. If so, we've found our spot.
## If not, we need to do another check

if len(check_trio_values_4['0'])==1 :
    print(f"FOUND! square needs 4: {check_trio_values_4['0'][0]}")
else:
    print('no final answer')

FOUND! square needs 4: (0, 1, 0)


In [27]:
trio_needs_3 = {key : val for key, val in b.board_map.items() if key[2] in [3] and key[1]==2}  # check for '3'
trio_needs_3

{(3, 2, 3): '0', (4, 2, 3): '0', (5, 2, 3): '2'}

In [28]:
invert_map(trio_needs_3)

{'0': [(3, 2, 3), (4, 2, 3)], '2': [(5, 2, 3)]}

In [ ]:
[val for key, val in b.board_map.items() if key[2]==0]

In [ ]:
{key : val for key, val in b.board_map.items() if val=='4'}

In [ ]:
b.forward_solve()
print(b)

In [ ]:
# b.set_square((0, 1), '4')
# b.set_square((6, 4), '5')
b.empty_squares[(0,1)] = set(['4'])
b.empty_squares[(6,4)] = set(['5'])
print(b)


In [ ]:
b.forward_solve()
print(b)

In [ ]:
b.empty_squares[(0,8)] = set(['6'])
b.empty_squares[(4,3)] = set(['4'])
b.empty_squares[(4,7)] = set(['2'])
b.forward_solve()
print(b)

In [ ]:
b.empty_squares[(3,4)] = set(['2'])
b.empty_squares[(4,6)] = set(['6'])
b.empty_squares[(8,0)] = set(['6'])
b.forward_solve()
print(b)

In [ ]:
b.empty_squares[(1,2)] = set(['9'])
b.empty_squares[(4,2)] = set(['3'])
b.empty_squares[(5,4)] = set(['3'])
b.empty_squares[(6,3)] = set(['6'])
b.forward_solve()
print(b)

In [ ]:
b.empty_squares[(5,1)] = set(['8'])
b.forward_solve()
print(b)

In [ ]:
for logline in b.solve_log:
    print(logline)

In [ ]:
b.forward_solve()
print(b)

In [ ]:
b.get_box_row(0)

In [ ]:
b.get_box_col(0)

In [ ]:
# box_col == 0 means the first column of boxes, i.e. the first 3 columns

digit_summary = hash_count(flatten(b.get_box_col(0)))
digit_summary.pop('0')
digit_summary
digits_filled_in_2_boxes_out_of_3 = {key : val for key, val in digit_summary.items() if val==2}
digits_filled_in_2_boxes_out_of_3

In [ ]:
b.get_cols_containing_digit('4')

In [ ]:
b.get_box_col(0)

In [ ]:
digit_summary_w_index = hash_count_bis(flatten(list(map(lambda x: list(enumerate(x)), b.get_box_col(0)))))
digit_summary_w_index.pop('0')
digit_summary_w_index
digits_filled_in_2_boxes_out_of_3_idx = {key : val for key, val in digit_summary_w_index.items() if len(val)==2}
digits_filled_in_2_boxes_out_of_3_idx

In [ ]:
b.get_cols_containing_digit('4')

In [ ]:
cols_in_box_col_0 = [0, 1, 2] 

In [ ]:
column_needs_digit4 = set(cols_in_box_col_0).difference(set(b.get_cols_containing_digit('4'))).pop()
column_needs_digit4

In [ ]:
b.get_rows_containing_digit('4')

In [ ]:
b.get_cols_containing_digit('4')

In [ ]:
b.get_boxes_containing_digit('4')

In [ ]:
boxes_in_box_col0  =[0, 3, 6]

In [ ]:
box_neeeds_digit4 = set(boxes_in_box_col0).difference(set(b.get_boxes_containing_digit('4'))).pop()
box_neeeds_digit4

In [ ]:
b.get_box(box_neeeds_digit4)

In [ ]:
b.get_col(column_needs_digit4)

In [ ]:
hash_count_bis(flatten(list(map(lambda x: list(enumerate(x)), b.get_box_col(0)))))

In [ ]:
list(map(lambda x: '2' in x, b.rows))

In [ ]:
[i for i, x in enumerate(map(lambda x: '2' in x, b.get_cols())) if x]


In [ ]:
list(filter(lambda x: '2' in x, b.rows))

In [ ]:
list(map(lambda x: x[0] if '2' in x[1] else None, enumerate(b.rows)))

In [ ]:
b.get_boxes_containing_digit('2')

In [ ]:
list(enumerate(b.rows))

In [ ]:
b.get_two_candidate_squares()

In [ ]:
b.set_square((0,3), 9)

In [ ]:
b.update_candidate_notes()

In [ ]:
b.empty_squares.pop((0,3))

In [ ]:
print(''.join(b.rows[0]))

In [ ]:
# print(f'{b.name}\n' + '\n'.join(list(map(lambda r: ''.join(r), b.rows))))
print(b.export_boardstring())

In [ ]:
#for sq in b.empty_squares:
#    b.calc_candidates(sq)
#b.update_candidate_notes()
hash_count(list(map(len, b.empty_squares.values())), sort=True)

In [ ]:
print(len(b.empty_squares))

In [ ]:
b.fill_in_solved_squares()

In [ ]:
print(b)

In [ ]:
b.empty_squares

In [ ]:
res = {key : val for key, val in b.empty_squares.items()
                   if len(val)==1}
res

In [ ]:
for chunk in boardchunks:
    b = Board(chunk)
    b.forward_solve()
    print(b.name)
    for logline in b.solve_log:
        print(logline)
    

In [ ]:
boards = {}

key = 'Grid 01'
b = []

boardlines = 'Grid 01\n'
with open(infile) as f:
    for line in f:
        linestr = line.rstrip()
#        print(linestr)
        
        if linestr.startswith('Grid'):
            boards[key] = b
            key = linestr
            b = []
        else:
            linelist = list(linestr)
            b.append(linelist)
#            print(linelist)
boards

In [ ]:
Box 0:
[self.rows[0][0:3], self.rows[1][0:3], self.rows[2][0:3] ]
Box 1:
[self.rows[0][3:6], self.rows[1][3:6], self.rows[2][3:6] ]

Box 2:
[self.rows[0][6:9], self.rows[1][6:9], self.rows[2][6:9] ]

Box 3:
[self.rows[3][0:3], self.rows[4][0:3], self.rows[5][0:3] ]

Box 6:
[self.rows[6][0:3], self.rows[7][0:3], self.rows[8][0:3] ]
Box 7:
[self.rows[6][3:6], self.rows[7][3:6], self.rows[8][3:6] ]


In [ ]:
for box in [0, 1, 2, 3, 4, 5, 6, 7, 8]:
#    print(f'box: {box} b_row:{box//3} b_col:{box%3}')
    print(f'({box//3}, {box%3}) : {box},')

In [ ]:
b.rows[0:3]

In [ ]:
b.rows

In [ ]:
b.get_box(3)

In [ ]:
list(map(lambda x: b.get_col(x), range(0,9)))

In [ ]:
b.rows

In [ ]:
b.get_rows()

In [ ]:
b.get_cols()

In [ ]:
b.get_boxes()

In [ ]:
for r in range(0,9):
    for c in range(0,9):
        print( (r,c, b.get_box_for_square( (r,c) )) )

In [ ]:
b.board_map

In [ ]:
j = invert_map(b.board_map)
j.pop('0')
j

In [29]:
t = '''
	<text x="200" y="200" class="note">1,1</text>
	<text x="300" y="200" class="note">1,2</text>
	<text x="200" y="300" class="note">2,1</text>
	<text x="300" y="300" class="note">2,2</text>
'''


In [36]:
svg_sq = '<text x="300" y="200" class="note">1,2</text>'
for r in range(3,9):
    for c in range(3,9):
        # print(r,c)
        svg_sq = f'<text x="{100*(c+1)}" y="{100*(r+1)}" class="note">{r},{c}</text>'
        print(svg_sq)
        

<text x="400" y="400" class="note">3,3</text>
<text x="500" y="400" class="note">3,4</text>
<text x="600" y="400" class="note">3,5</text>
<text x="700" y="400" class="note">3,6</text>
<text x="800" y="400" class="note">3,7</text>
<text x="900" y="400" class="note">3,8</text>
<text x="400" y="500" class="note">4,3</text>
<text x="500" y="500" class="note">4,4</text>
<text x="600" y="500" class="note">4,5</text>
<text x="700" y="500" class="note">4,6</text>
<text x="800" y="500" class="note">4,7</text>
<text x="900" y="500" class="note">4,8</text>
<text x="400" y="600" class="note">5,3</text>
<text x="500" y="600" class="note">5,4</text>
<text x="600" y="600" class="note">5,5</text>
<text x="700" y="600" class="note">5,6</text>
<text x="800" y="600" class="note">5,7</text>
<text x="900" y="600" class="note">5,8</text>
<text x="400" y="700" class="note">6,3</text>
<text x="500" y="700" class="note">6,4</text>
<text x="600" y="700" class="note">6,5</text>
<text x="700" y="700" class="note"

In [39]:
for sq in b.board_map:
    # print(sq)
    if b.board_map[sq] != '0':
        (row,col,box) = sq
        svg_sq = f'<text x="{100*(col+1)}" y="{100*(row+1)}">{b.board_map[sq]}</text>'
        print(svg_sq)

<text x="100" y="100">2</text>
<text x="500" y="100">8</text>
<text x="700" y="100">3</text>
<text x="200" y="200">6</text>
<text x="500" y="200">7</text>
<text x="800" y="200">8</text>
<text x="900" y="200">4</text>
<text x="200" y="300">3</text>
<text x="400" y="300">5</text>
<text x="700" y="300">2</text>
<text x="900" y="300">9</text>
<text x="400" y="400">1</text>
<text x="600" y="400">5</text>
<text x="700" y="400">4</text>
<text x="900" y="400">8</text>
<text x="100" y="600">4</text>
<text x="300" y="600">2</text>
<text x="400" y="600">7</text>
<text x="600" y="600">6</text>
<text x="100" y="700">3</text>
<text x="300" y="700">1</text>
<text x="600" y="700">7</text>
<text x="800" y="700">4</text>
<text x="100" y="800">7</text>
<text x="200" y="800">2</text>
<text x="500" y="800">4</text>
<text x="800" y="800">6</text>
<text x="300" y="900">4</text>
<text x="500" y="900">1</text>
<text x="900" y="900">3</text>


In [40]:
b.name

'Grid 02'

In [ ]:
</g>

</svg>


In [45]:
with open('test_out.svg', 'w') as outfile:
    with open('board_template_basefile.svg', 'r') as infile:
        outfile.write(infile.read())
        for sq in b.board_map:
            # print(sq)
            if b.board_map[sq] != '0':
                (row,col,box) = sq
                svg_sq = f'<text x="{100*(col+1)}" y="{100*(row+1)}">{b.board_map[sq]}</text>'
                outfile.write(svg_sq)
        outfile.write('</g></svg>')

In [91]:
b.export_svg_board(show_notes=True)

In [58]:
d={}
for e in b.empty_squares[0,1]:
    d[e]=e

In [67]:
n = '''
<text x="200" y="60" class="note">1	&#160; &#160;</text>
<text x="200" y="90" class="note">4 5 &#160;</text>
<text x="200" y="120" class="note">7 &#160; 9</text>
'''
(row, col) = (0,1)
n = f'''
<text x="{100*(col+1)}" y="{100*(row+1)-40}" class="note">{d.get('1', '&#160;')} {d.get('2', '&#160;')} {d.get('3', '&#160;')}</text>
<text x="{100*(col+1)}" y="{100*(row+1)-10}" class="note">{d.get('4', '&#160;')} {d.get('5', '&#160;')} {d.get('6', '&#160;')}</text>
<text x="{100*(col+1)}" y="{100*(row+1)+20}" class="note">{d.get('7', '&#160;')} {d.get('8', '&#160;')} {d.get('9', '&#160;')}</text>
'''


In [68]:
print(n)


<text x="200" y="60" class="note">1 &#160; &#160;</text>
<text x="200" y="90" class="note">4 5 &#160;</text>
<text x="200" y="120" class="note">7 &#160; 9</text>



In [127]:
b.export_svg_board(filename='Grid 02 pre-solve.svg', show_notes=True)

In [128]:
b.forward_solve()

In [130]:
b.export_svg_board(filename='Grid 02 post-solve.svg', show_notes=True)

KeyError: (2, 4)

In [129]:
print(b)

Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '0', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [131]:
b.empty_squares

{(0, 1): {'1', '4', '5', '7', '9'},
 (0, 2): {'5', '7', '9'},
 (0, 3): {'4', '6', '9'},
 (0, 5): {'1', '4', '9'},
 (0, 7): {'1', '5', '7'},
 (0, 8): {'1', '5', '6', '7'},
 (1, 0): {'1', '5', '9'},
 (1, 2): {'5', '9'},
 (1, 3): {'2', '3', '9'},
 (1, 5): {'1', '2', '3', '9'},
 (1, 6): {'1', '5'},
 (2, 0): {'1', '8'},
 (2, 2): {'7', '8'},
 (2, 5): {'1', '4'},
 (2, 7): {'1', '7'},
 (3, 0): {'6', '9'},
 (3, 1): {'7', '9'},
 (3, 2): {'3', '6', '7', '9'},
 (3, 4): {'2', '3', '9'},
 (3, 7): {'2', '3', '7', '9'},
 (4, 0): {'1', '5', '6', '8', '9'},
 (4, 1): {'1', '5', '7', '8', '9'},
 (4, 2): {'3', '5', '6', '7', '8', '9'},
 (4, 3): {'2', '3', '4', '8', '9'},
 (4, 4): {'2', '3', '9'},
 (4, 5): {'2', '3', '4', '8', '9'},
 (4, 6): {'1', '5', '6', '7', '9'},
 (4, 7): {'1', '2', '3', '5', '7', '9'},
 (4, 8): {'1', '2', '5', '6', '7'},
 (5, 1): {'1', '5', '8', '9'},
 (5, 4): {'3', '9'},
 (5, 6): {'1', '5', '9'},
 (5, 7): {'1', '3', '5', '9'},
 (5, 8): {'1', '5'},
 (6, 1): {'5', '8', '9'},
 (6, 3): {

In [133]:
print(b)

Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '0', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]


In [134]:
b.get_solved_squares()

{}

In [209]:
b.board_map

{(0, 0, 0): '2',
 (0, 1, 0): '0',
 (0, 2, 0): '0',
 (0, 3, 1): '0',
 (0, 4, 1): '8',
 (0, 5, 1): '0',
 (0, 6, 2): '3',
 (0, 7, 2): '0',
 (0, 8, 2): '0',
 (1, 0, 0): '0',
 (1, 1, 0): '6',
 (1, 2, 0): '0',
 (1, 3, 1): '0',
 (1, 4, 1): '7',
 (1, 5, 1): '0',
 (1, 6, 2): '0',
 (1, 7, 2): '8',
 (1, 8, 2): '4',
 (2, 0, 0): '0',
 (2, 1, 0): '3',
 (2, 2, 0): '0',
 (2, 3, 1): '5',
 (2, 4, 1): '6',
 (2, 5, 1): '0',
 (2, 6, 2): '2',
 (2, 7, 2): '0',
 (2, 8, 2): '9',
 (3, 0, 3): '0',
 (3, 1, 3): '0',
 (3, 2, 3): '0',
 (3, 3, 4): '1',
 (3, 4, 4): '0',
 (3, 5, 4): '5',
 (3, 6, 5): '4',
 (3, 7, 5): '0',
 (3, 8, 5): '8',
 (4, 0, 3): '0',
 (4, 1, 3): '0',
 (4, 2, 3): '0',
 (4, 3, 4): '0',
 (4, 4, 4): '0',
 (4, 5, 4): '0',
 (4, 6, 5): '0',
 (4, 7, 5): '0',
 (4, 8, 5): '0',
 (5, 0, 3): '4',
 (5, 1, 3): '0',
 (5, 2, 3): '2',
 (5, 3, 4): '7',
 (5, 4, 4): '0',
 (5, 5, 4): '6',
 (5, 6, 5): '0',
 (5, 7, 5): '0',
 (5, 8, 5): '0',
 (6, 0, 6): '3',
 (6, 1, 6): '0',
 (6, 2, 6): '1',
 (6, 3, 7): '0',
 (6, 4, 7): '0

In [211]:
[b.get_row(r) for r in range(0,9)]

[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]

In [212]:
print(b)

Grid 02
[['2', '0', '0', '0', '8', '0', '3', '0', '0'],
 ['0', '6', '0', '0', '7', '0', '0', '8', '4'],
 ['0', '3', '0', '5', '6', '0', '2', '0', '9'],
 ['0', '0', '0', '1', '0', '5', '4', '0', '8'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['4', '0', '2', '7', '0', '6', '0', '0', '0'],
 ['3', '0', '1', '0', '0', '7', '0', '4', '0'],
 ['7', '2', '0', '0', '4', '0', '0', '6', '0'],
 ['0', '0', '4', '0', '1', '0', '0', '0', '3']]
